In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

pd.set_option("max_columns", 200)

In [2]:
columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]
columns3 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "PreClosePx", "openPrice", "HighPx", "LowPx", "close", "NUMTRADES", "cum_volume",
           "cum_amount", "TOTALLONGPOSITION", "PERATIO1", "PERATIO2", "ENDOFDAYMAKER", "TOTALOFFERQTY", "WEIGHTEDAVGOFFERPX", "TOTALBIDQTY", "WEIGHTEDAVGBIDPX"]
columns4 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "ask10p", "ask9p", "ask8p", "ask7p", "ask6p", "ask5p", "ask4p", "ask3p", "ask2p",
           "ask1p", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid6p", "bid7p", "bid8p", "bid9p", "bid10p", "ask10q", "ask9q", "ask8q", "ask7q",
           "ask6q", "ask5q", "ask4q", "ask3q", "ask2q", "ask1q", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "bid6q", "bid7q", "bid8q", "bid9q", 
           "bid10q", "NUMORDERS_B1", "NOORDERS _B1", "ORDERQTY_B1", "NUMORDERS_S1", "NOORDERS _S1", "ORDERQTY_S1"]
columns5 =  ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition"]


year = "2016"
df = []
bad = []
readPath = 'J:\\LEVEL2_shenzhen\\' + year + '\\03\\29'
dataPathLs = np.array(glob.glob(readPath))

for data in dataPathLs:
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue
    
    if data[-5:-3] + data[-2:] == '0413':
        bad.append(data)
        continue
    
    print('---------------------------------------------------------------------------------------------')
    print(data)
    # SZ version A without AM PM, 未分卷
    if len(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z')[0]).split('.')[0].split('_')[2]
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        a.close()
        
        snapshot1 = pd.DataFrame()
        snapshot2 = pd.DataFrame()
        if len(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***.txt'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\SZL2_SNAPSHOTDW_***')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        
        
        if len(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***.txt'))) != 2:
            path = r'F:\SZ\2016' 
            n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***.7z')[0]).split('.')[0]
            try:
                a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
            except:
                print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                print(data + "\\" + n_ame + '.7z')
                bad.append(data + "\\" + n_ame + '.7z')
                continue
            path1 = path + '\\' + d_ate
            a.extractall(path = path1)
            a.close()

            
        for pp in np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***.txt')):
            s1 = pd.read_table(pp, header=None)
            assert(s1.shape[1] == len(columns3))
            s1.columns = columns3
            snapshot1 = pd.concat([snapshot1, s1])
        del s1
        for pp in np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***.txt')):
            s2 = pd.read_table(pp, header=None)
            assert(s2.shape[1] == len(columns4))
            s2.columns = columns4
            snapshot2 = pd.concat([snapshot2, s2])
        del s2
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1 = snapshot1[snapshot1["DATATIMESTAMP"] != 0]
        snapshot2 = snapshot2[snapshot2["DATATIMESTAMP"] != 0]
        snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"], how="outer")
        try:
            assert((snapshot1.shape[0] == snapshot.shape[0]) & (snapshot2.shape[0] == snapshot.shape[0]))
        except:
            if snapshot1.shape[0] == snapshot.shape[0]:
                print("snapshot have more ticks than snapshotdw")
                tt = snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)]["DATATIMESTAMP"].unique()
                print(tt)
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"].isin(tt))].groupby("DATATIMESTAMP")["ask1p"].size())
                print(np.sort(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)]["DATATIMESTAMP"].unique()))
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)].shape[0])
                print((snapshot.shape[0] - snapshot2.shape[0])/snapshot.shape[0])
            elif snapshot2.shape[0] == snapshot.shape[0]:
                print("snapshotdw have more ticks than snapshot")
                print(snapshot[snapshot["close"].isnull()]["DATATIMESTAMP"].unique().min())
                print((snapshot.shape[0] - snapshot1.shape[0])/snapshot.shape[0])
            else:
                print("snapshotdw don't join with snapshot")
            snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"])
        del snapshot1
        del snapshot2
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot.loc[snapshot["openPrice"] == 0, "LowPx"] = 0
        
        print("SZ version A without AM PM，未分卷")
        
    # SZ version A without AM PM, 分卷
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z.001'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z.001')[0]).split('.')[0].split('_')[2]
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z.001')[0]).split('.')[0]
        os.system("copy /b " + n_ame + ".7z.* " + n_ame + ".7z")
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        a.close()
        
        snapshot1 = pd.DataFrame()
        snapshot2 = pd.DataFrame()
        if len(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***.txt'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\SZL2_SNAPSHOTDW_***')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        
        
        elif len(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***.txt'))) != 2:
            n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***.7z')[0]).split('.')[0]
            try:
                a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
            except:
                print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                print(data + "\\" + n_ame + '.7z')
                bad.append(data + "\\" + n_ame + '.7z')
                continue
            path1 = path + '\\' + d_ate
            a.extractall(path = path1)
            a.close()
        
        
        
        for pp in np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***.txt')):
            s1 = pd.read_table(pp, header=None)
            assert(s1.shape[1] == len(columns3))
            s1.columns = columns3
            snapshot1 = pd.concat([snapshot1, s1])
        del s1
        for pp in np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***.txt')):
            s2 = pd.read_table(pp, header=None)
            assert(s2.shape[1] == len(columns4))
            s2.columns = columns4
            snapshot2 = pd.concat([snapshot2, s2])
        del s2
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1 = snapshot1[snapshot1["DATATIMESTAMP"] != 0]
        snapshot2 = snapshot2[snapshot2["DATATIMESTAMP"] != 0]
        snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"], how="outer")
        try:
            assert((snapshot1.shape[0] == snapshot.shape[0]) & (snapshot2.shape[0] == snapshot.shape[0]))
        except:
            if snapshot1.shape[0] == snapshot.shape[0]:
                print("snapshot have more ticks than snapshotdw")
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)]["DATATIMESTAMP"].unique())
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)].shape[0])
                print(np.sort(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)]["DATATIMESTAMP"].unique()))
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)].shape[0])
                print((snapshot.shape[0] - snapshot2.shape[0])/snapshot.shape[0])
            elif snapshot2.shape[0] == snapshot.shape[0]:
                print("snapshotdw have more ticks than snapshot")
                print(snapshot[snapshot["close"].isnull()]["DATATIMESTAMP"].unique().min())
                print((snapshot.shape[0] - snapshot1.shape[0])/snapshot.shape[0])
            else:
                print("snapshotdw don't join with snapshot")
            snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"])
        del snapshot1
        del snapshot2
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot.loc[snapshot["openPrice"] == 0, "LowPx"] = 0
        
        print("SZ version A without AM PM，分卷")
        
    # SZ version A AM PM
    # I:\\LEVEL2_shenzhen\\2016\\04\\19\\SZL2_SNAPSHOT_20160419_AM.7z 58
    # I:\\LEVEL2_shenzhen\\2016\\05\\06\\SZL2_SNAPSHOT_20160506_AM.txt 59
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_AM.txt'))) == 1:
        snap1 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns4))
        snap1.columns = columns4
        snap2 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns4))
        snap2.columns = columns4
        snapshot2 = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        del snap1
        del snap2
        
        snap1 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns3))
        snap1.columns = columns3
        snap2 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns3))
        snap2.columns = columns3
        snapshot1 = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        del snap1
        del snap2
        
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1 = snapshot1[snapshot1["DATATIMESTAMP"] != 0]
        snapshot2 = snapshot2[snapshot2["DATATIMESTAMP"] != 0]
        snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"], how="outer")
        
        try:
            assert((snapshot1.shape[0] == snapshot.shape[0]) & (snapshot2.shape[0] == snapshot.shape[0]))
        except:
            if snapshot1.shape[0] == snapshot.shape[0]:
                print("snapshot have more ticks than snapshotdw")
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)]["DATATIMESTAMP"].unique())
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)].shape[0])
                print(np.sort(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)]["DATATIMESTAMP"].unique()))
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)].shape[0])
                print((snapshot.shape[0] - snapshot2.shape[0])/snapshot.shape[0])
            elif snapshot2.shape[0] == snapshot.shape[0]:
                print("snapshotdw have more ticks than snapshot")
                print(snapshot[snapshot["close"].isnull()]["DATATIMESTAMP"].unique().min())
                print((snapshot.shape[0] - snapshot1.shape[0])/snapshot.shape[0])
            else:
                print("snapshotdw don't join with snapshot")
            snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"])
            
        del snapshot1
        del snapshot2
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot.loc[snapshot["openPrice"] == 0, "LowPx"] = 0
        
        print("SZ version A AM PM, txt")
                
        
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_AM.7z'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_AM.7z')[0]).split('.')[0].split('_')[2]
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_AM.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        
        n_ame1 = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_PM.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame1 + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame1 + '.7z')
            bad.append(data + "\\" + n_ame1 + '.7z')
            continue
        a.extractall(path = path1)
        
        if len(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\SZL2_SNAPSHOT_***')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        snap1 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns3))
        snap1.columns = columns3
        snap2 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns3))
        snap2.columns = columns3
        snapshot1 = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        del snap1
        del snap2
        
        snap1 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns4))
        snap1.columns = columns4
        snap2 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns4))
        snap2.columns = columns4
        snapshot2 = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        del snap1
        del snap2
        
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1 = snapshot1[snapshot1["DATATIMESTAMP"] != 0]
        snapshot2 = snapshot2[snapshot2["DATATIMESTAMP"] != 0]
        snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"], how="outer")
        
        try:
            assert((snapshot1.shape[0] == snapshot.shape[0]) & (snapshot2.shape[0] == snapshot.shape[0]))
        except:
            if snapshot1.shape[0] == snapshot.shape[0]:
                print("snapshot have more ticks than snapshotdw")
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)]["DATATIMESTAMP"].unique())
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)].shape[0])
                print(np.sort(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)]["DATATIMESTAMP"].unique()))
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)].shape[0])
                print((snapshot.shape[0] - snapshot2.shape[0])/snapshot.shape[0])
            elif snapshot2.shape[0] == snapshot.shape[0]:
                print("snapshotdw have more ticks than snapshot")
                print(snapshot[snapshot["close"].isnull()]["DATATIMESTAMP"].unique().min())
                print((snapshot.shape[0] - snapshot1.shape[0])/snapshot.shape[0])
            else:
                print("snapshotdw don't join with snapshot")
            snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"])
        del snapshot1
        del snapshot2
        
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot.loc[snapshot["openPrice"] == 0, "LowPx"] = 0
        
        print("SZ version A AM PM, .7z")
 
    # am & pm
    # 'I:\\LEVEL2_shenzhen\\2016\\05\\09\\am_snap_level_spot.7z'
    # 'I:\\LEVEL2_shenzhen\\2016\\05\\10\\am_snap_level_spot.7z.001'
    
    elif len(np.array(glob.glob(data +'\\am_snap_level_spot.7z'))) == 1:
        date = glob.glob(data +'\\am_snap_level_spot.7z')[0][19:23] + glob.glob(data +'\\am_snap_level_spot.7z')[0][24:26] + glob.glob(data +'\\am_snap_level_spot.7z')[0][27:29]
        path = r'F:\SZ\2016' 
        os.chdir(data)
        try:
            a = py7zr.SevenZipFile(data + '\\am_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_snap_level_spot.7z')
            bad.append(data + '\\am_snap_level_spot.7z')
            continue
        path1 = path + '\\' + date
        a.extractall(path = path1)
        a.close()
        try:
            a = py7zr.SevenZipFile(data + '\\pm_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_snap_level_spot.7z')
            bad.append(data + '\\pm_snap_level_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
        
        if len(np.array(glob.glob(path1 +'\\***_hq_snap_spot.txt'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\***_hq_snap_spot.txt')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        try:
            assert(am_snap1.shape[1] == len(columns1))
            am_snap1.columns = columns1       
        except:
            assert(am_snap1.shape[1] == len(columns5))
            am_snap1.columns = columns5     
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        try:
            assert(pm_snap1.shape[1] == len(columns1))
            pm_snap1.columns = columns1       
        except:
            assert(pm_snap1.shape[1] == len(columns5))
            pm_snap1.columns = columns5     
        snapshot1 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        am_snap1 = pd.read_table(path1 + "\\am_snap_level_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns2))
        am_snap1.columns = columns2       
        pm_snap1 = pd.read_table(path1 + "\\pm_snap_level_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns2))
        pm_snap1.columns = columns2       
        snapshot2 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1['time'] = (snapshot1['OrigTime'] - int(snapshot1['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot2['time'] = (snapshot2['OrigTime'] - int(snapshot2['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"], how="outer")
        try:
            assert((snapshot.shape[0] == snapshot1.shape[0]) & (snapshot.shape[0] == snapshot2.shape[0]))
        except:
            if snapshot.shape[0] == snapshot1.shape[0]:
                print("snapshot1 have more ticks than snapshot2")
                if all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            elif snapshot.shape[0] == snapshot2.shape[0]:
                print("snapshot2 have more ticks than snapshot1")
                if all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            else:
                print("snapshot2 don't join with snapshot1")
            snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"])
        del snapshot1
        del snapshot2
           
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 未分卷")
        
    elif len(np.array(glob.glob(data +'\\am_snap_level_spot.7z.001'))) == 1:    
        date = glob.glob(data +'\\am_snap_level_spot.7z.001')[0][19:23] + glob.glob(data +'\\am_snap_level_spot.7z.001')[0][24:26] + glob.glob(data +'\\am_snap_level_spot.7z.001')[0][27:29]
        path = r'F:\SZ\2016' 
        os.chdir(data)
        os.system("copy /b am_snap_level_spot.7z.* am_snap_level_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\am_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_snap_level_spot.7z')
            bad.append(data + '\\am_snap_level_spot.7z')
            continue
        path1 = path + '\\' + date
        a.extractall(path = path1)
        a.close()
        
        os.system("copy /b pm_snap_level_spot.7z.* pm_snap_level_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\pm_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_snap_level_spot.7z')
            bad.append(data + '\\pm_snap_level_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
        
        if len(np.array(glob.glob(path1 +'\\***_hq_snap_spot.txt'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\***_hq_snap_spot.txt')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        try:
            assert(am_snap1.shape[1] == len(columns1))
            am_snap1.columns = columns1       
        except:
            assert(am_snap1.shape[1] == len(columns5))
            am_snap1.columns = columns5       
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        try:
            assert(pm_snap1.shape[1] == len(columns1))
            pm_snap1.columns = columns1       
        except:
            assert(pm_snap1.shape[1] == len(columns5))
            pm_snap1.columns = columns5     
        snapshot1 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        am_snap1 = pd.read_table(path1 + "\\am_snap_level_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns2))
        am_snap1.columns = columns2        
        pm_snap1 = pd.read_table(path1 + "\\pm_snap_level_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns2))
        pm_snap1.columns = columns2       
        snapshot2 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1['time'] = (snapshot1['OrigTime'] - int(snapshot1['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot2['time'] = (snapshot2['OrigTime'] - int(snapshot2['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"], how="outer")
        try:
            assert((snapshot.shape[0] == snapshot1.shape[0]) & (snapshot.shape[0] == snapshot2.shape[0]))
        except:
            if snapshot.shape[0] == snapshot1.shape[0]:
                print("snapshot1 have more ticks than snapshot2")
                if all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            elif snapshot.shape[0] == snapshot2.shape[0]:
                print("snapshot2 have more ticks than snapshot1")
                if all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            else:
                print("snapshot2 don't join with snapshot1")
            snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"])
        del snapshot1
        del snapshot2
        
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 分卷") 
    else:
        print("break!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        break
        
        
    snapshot = snapshot[(snapshot["StockID"] < 4000) | (snapshot["StockID"] > 300000)]
    snapshot["cum_max"] = snapshot.groupby("StockID")["cum_volume"].transform(max)
    s2 = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID").first().reset_index()
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: str(x//1000000000))
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
    s2 = s2.rename(columns={"StockID": "ID", "OrigTime":"date", 'openPrice':"d_open", "PreClosePx":"d_yclose","HighPx":"d_high", "LowPx":"d_low", "close":"d_close", "cum_volume":"d_volume", "cum_amount":"d_amount"})
    s2 = s2[["ID", "date", "d_open", "d_yclose", "d_high", "d_low", "d_close", "d_volume", "d_amount"]]
    
    # 1. check massive missing   
    date = pd.DataFrame(pd.date_range(start='2019-06-10 08:30:00', end='2019-06-10 18:00:00', freq='s'), columns=["Orig"])
    date["time"] = date["Orig"].apply(lambda x: int(x.strftime("%H%M%S"))*1000)
    date["group"] = date["time"]//30000
    snapshot["group"] = snapshot["time"]//30000
    gl = date[((date["time"] >= 93000000) & (date["time"] <= 113000000))|((date["time"] >= 130000000) & (date["time"] <= 145700000))]["group"].unique()
    print("massive missing")
    print(set(gl) - set(snapshot["group"].unique()))
    
 
    # 2. check each security missing
#     snapshot = snapshot.sort_values(by=["StockID", "OrigTime"])
#     dd = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID")["time"].first().reset_index()
    
#     data2 = snapshot[(snapshot["time"] > 92500000) & (snapshot["time"] <= 145700000) & (snapshot["cum_volume"] > 0)][["StockID", "cum_volume",
#                                                                                                                      "OrigTime", "time"]]
#     del snapshot
#     data2 = data2.sort_values(by=["StockID", "OrigTime"])    
#     data2["volume_update"] = data2.groupby("StockID")["cum_volume"].apply(lambda x: x - x.shift(1))
#     data2["time1"] = pd.to_datetime(data2["OrigTime"].apply(lambda x: str(x)[:-3]))
#     data2["time_interval"] = data2.groupby("StockID")["time1"].apply(lambda x: x - x.shift(1))
#     data2["time_interval"] = data2["time_interval"].apply(lambda x: x.seconds)
#     data2 = pd.merge(data2, data2[data2["volume_update"] > 0].groupby("StockID")["time_interval"].describe()["75%"].reset_index(), on="StockID")
#     data2 = data2.rename(columns={"75%": "75%_time"})
#     vu = data2[data2["volume_update"] > 0].groupby("StockID")["volume_update"].apply(lambda x: x.describe([0.9])["90%"]).reset_index()
#     vu = vu.rename(columns={"volume_update":"90%"})
#     data2 = pd.merge(data2, vu, on="StockID")
#     data2["90%"] = data2["90%"].apply(lambda x: max(1000, x))
#     data2["indicator"] = 0
#     data2.loc[data2[data2["time"] >= 130000000].groupby("StockID").head(1).index, 'indicator'] = 1
#     data2.loc[data2[data2["time"] >= 93000000].groupby("StockID").head(1).index, 'indicator'] = 1
#     data2 = data2[data2["indicator"] != 1]
#     print("single security missing")
#     print(data2[(data2["time_interval"] > 180) & (data2["volume_update"] >= data2["90%"]) & (data2["75%_time"] <= 4)])



---------------------------------------------------------------------------------------------
J:\LEVEL2_shenzhen\2016\03\29


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


snapshot have more ticks than snapshotdw
[]
Series([], Name: ask1p, dtype: int64)
[145644820 145652050 145653390 ... 145959120 145959810 145959860]
5457
0.0007027757129367912
SZ version A without AM PM，未分卷
massive missing
{4681, 4683, 4684, 4685, 4686}


In [110]:
l = np.sort(list(set(gl) - set(snapshot["group"].unique())))

In [111]:
snapshot["has_missing1"] = 0 

In [112]:
for i in range(len(l)):
    snapshot["t"] = snapshot[snapshot["group"] > l[i]].groupby("StockID")["time"].transform("min")
    snapshot["has_missing1"] = np.where(snapshot["time"] == snapshot["t"], 1, 0)

In [124]:
kk = snapshot[snapshot["has_missing1"] == 1].groupby("StockID")["close"].size().reset_index()
kk[kk["close"] != 1]

,StockID,close
5,10,2
8,14,3
17,27,2
25,37,4
29,43,3
...,...,...
1515,300484,2
1516,300485,3
1518,300487,8
1523,300492,3


In [125]:
snapshot["time_dup"] = snapshot.groupby(["StockID", "time"])

,Date,SENDTIME,StockID,DATATIMESTAMP,PreClosePx,openPrice,HighPx,LowPx,close,NUMTRADES,cum_volume,cum_amount,TOTALLONGPOSITION,PERATIO1,PERATIO2,ENDOFDAYMAKER,TOTALOFFERQTY,WEIGHTEDAVGOFFERPX,TOTALBIDQTY,WEIGHTEDAVGBIDPX,ask10p,ask9p,ask8p,ask7p,ask6p,ask5p,ask4p,ask3p,ask2p,ask1p,bid1p,bid2p,bid3p,bid4p,bid5p,bid6p,bid7p,bid8p,bid9p,bid10p,ask10q,ask9q,ask8q,ask7q,ask6q,ask5q,ask4q,ask3q,ask2q,ask1q,bid1q,bid2q,bid3q,bid4q,bid5q,bid6q,bid7q,bid8q,bid9q,bid10q,NUMORDERS_B1,NOORDERS _B1,ORDERQTY_B1,NUMORDERS_S1,NOORDERS _S1,ORDERQTY_S1,time,OrigTime,cum_max,group,time_diff,volume_update,datetime,time_interval,time1,time2,time3,time4,tn_update,99%,95%,90%,has_missing1,t0,t1,t2,t3,t4
29151,20160329,140740,10,140735040,8.5,8.48,8.53,8.12,8.23,4987,9706962,80565185.37,0,602.84,0.0,N,2618708,8.78,1556364,8.02,8.34,8.33,8.32,8.31,8.3,8.29,8.28,8.26,8.25,8.24,8.23,8.22,8.21,8.2,8.19,8.18,8.17,8.16,8.15,8.14,1000,47700,50400,25500,10700,38600,6300,6300,43700,4400,41100,14200,17800,28600,20200,49500,18400,46700,116164,11800,1,1,41100,1,1,1000,140735040,20160329140735040,11854397,4691,324710.0,167100.0,2016-03-29 14:07:35,205.0,93000400,130000200,150019840.0,140735040,58.0,24.84,8.0,5.4,1,140735040.0,140735040.0,140735040.0,140735040.0,140735040.0
29152,20160329,140743,10,140735040,8.5,8.48,8.53,8.12,8.23,4987,9706962,80565185.37,0,602.84,0.0,N,2618708,8.78,1556364,8.02,8.34,8.33,8.32,8.31,8.3,8.29,8.28,8.26,8.25,8.24,8.23,8.22,8.21,8.2,8.19,8.18,8.17,8.16,8.15,8.14,1000,47700,50400,25500,10700,38600,6300,6300,43700,4400,41100,14200,17800,28600,20200,49500,18400,46700,116164,11800,1,1,41100,1,1,1000,140735040,20160329140735040,11854397,4691,0.0,0.0,2016-03-29 14:07:35,0.0,93000400,130000200,150019840.0,140735040,0.0,24.84,8.0,5.4,1,140735040.0,140735040.0,140735040.0,140735040.0,140735040.0


In [115]:
tr.shape[0]

1537

In [ ]:
snapshot["has_missing"] = np.where(snapshot[]snapshot[snapshot["group"] > i].groupby("StockID")["time"].min()

In [ ]:
[1,2,3,5,6,7]
[1,2,6,7]

In [279]:
snapshot[(snapshot["time"] < 105158000)]["time"].max()

105157000

In [5]:
snapshot["time_diff"] = snapshot.groupby("StockID")["time"].apply(lambda x: x-x.shift(1))
snapshot["volume_update"] = snapshot.groupby("StockID")["cum_volume"].apply(lambda x: x-x.shift(1))
snapshot["datetime"] = pd.to_datetime(snapshot["OrigTime"].apply(lambda x: str(x)[:-3]))
snapshot["time_interval"] = snapshot.groupby("StockID")["datetime"].apply(lambda x: x - x.shift(1))
snapshot["time_interval"] = snapshot["time_interval"].apply(lambda x: x.seconds)
snapshot["tn_update"] = snapshot.groupby("StockID")["NUMTRADES"].apply(lambda x: x-x.shift(1))

f1 = snapshot[(snapshot["time"] > 93000000) & (snapshot["volume_update"] != 0)].groupby("StockID")["time"].min().reset_index()
f1 = f1.rename(columns={"time": "time1"})
f2 = snapshot[(snapshot["time"] >= 130000000) & (snapshot["volume_update"] != 0)].groupby("StockID")["time"].min().reset_index()
f2 = f2.rename(columns={"time": "time2"})
f3 = snapshot[(snapshot["time"] > 150000000) & (snapshot["volume_update"] != 0)].groupby("StockID")["time"].min().reset_index()
f3 = f3.rename(columns={"time": "time3"})
snapshot = pd.merge(snapshot, f1, on="StockID", how="outer")
snapshot = pd.merge(snapshot, f2, on="StockID", how="outer")
snapshot = pd.merge(snapshot, f3, on="StockID", how="outer")

In [6]:
f4 = snapshot[(snapshot["time"] > 140414000) & (snapshot["volume_update"] != 0)].groupby("StockID")["time"].min().reset_index()
f4 = f4.rename(columns={"time": "time4"})
# f5 = snapshot[(snapshot["time"] > 100623000) & (snapshot["volume_update"] != 0)].groupby("StockID")["time"].min().reset_index()
# f5 = f5.rename(columns={"time": "time5"})
snapshot = pd.merge(snapshot, f4, on="StockID", how="outer")
# snapshot = pd.merge(snapshot, f5, on="StockID", how="outer")
tr = snapshot.groupby("StockID")["time4"].first().reset_index()
display(tr.head(20))

,StockID,time4
0,1,140735860
1,5,140735370
2,6,140735090
3,8,140735800
4,9,140737260
5,10,140735040
6,11,140732950
7,12,140732040
8,14,140734810
9,17,140734110


In [91]:
p99 = snapshot[(snapshot["time"] > 93000000) & (snapshot["time"] < 145700000) & (snapshot["time"] != snapshot["time2"]) &
              (snapshot["tn_update"] != 0)]\
.groupby("StockID")["tn_update"].apply(lambda x: x.describe([0.99])["99%"]).reset_index()
p90 = snapshot[(snapshot["time"] > 93000000) & (snapshot["time"] < 145700000) & (snapshot["time"] != snapshot["time2"]) & (snapshot["tn_update"] != 0)]\
.groupby("StockID")["tn_update"].apply(lambda x: x.describe([0.9])["90%"]).reset_index()
p95 = snapshot[(snapshot["time"] > 93000000) & (snapshot["time"] < 145700000) & (snapshot["time"] != snapshot["time2"]) & (snapshot["tn_update"] != 0)]\
.groupby("StockID")["tn_update"].apply(lambda x: x.describe([0.95])["95%"]).reset_index()

In [99]:
snapshot[(snapshot["StockID"] == 36) & (snapshot["tn_update"] != 0)]["tn_update"].describe([0.9, 0.95, 0.99])

count    2198.000000
mean        3.173339
std         6.773968
min         1.000000
50%         2.000000
90%         6.000000
95%         9.000000
99%        30.060000
max       138.000000
Name: tn_update, dtype: float64

In [92]:
snapshot.drop("99%",axis=1,inplace=True)
snapshot.drop("95%",axis=1,inplace=True)
snapshot.drop("90%",axis=1,inplace=True)
p99 = p99.rename(columns={"tn_update":"99%"})
p95 = p95.rename(columns={"tn_update":"95%"})
p90 = p90.rename(columns={"tn_update":"90%"})
snapshot = pd.merge(snapshot, p99, on="StockID", how="left")
snapshot = pd.merge(snapshot, p95, on="StockID", how="left")
snapshot = pd.merge(snapshot, p90, on="StockID", how="left")

In [107]:
re = snapshot[(snapshot["time_interval"] > 60) & (snapshot["tn_update"] > snapshot["90%"]) & 
         (snapshot["time"] > snapshot["time1"]) & (snapshot["time"] != snapshot["time2"]) & 
         (snapshot["time"] != snapshot["time3"])][["StockID", "time", "tn_update", "90%", "95%", "99%", "time_interval"]]
re["size"] = re.groupby("StockID")["time"].transform("nunique")
display(re[re["size"] != 1])
display(len(re["StockID"].unique()))
display(tr.shape[0])
display(tr.shape[0] - len(re["StockID"].unique()))

,StockID,time,tn_update,90%,95%,99%,time_interval,size


1529

1537

8

In [101]:
set(tr["StockID"]) - set(re["StockID"])

{828,
 877,
 2082,
 2110,
 2394,
 2459,
 2534,
 2725,
 2791,
 2792,
 300238,
 300417,
 300484,
 300506}

In [71]:
kk = pd.merge(tr, re[re["size"] == 1], on="StockID", how="left")
kk[kk["time"] != kk["time4"]]

,StockID,time4,time,volume_update,90%,95%,99%,time_interval,size
475,2082,140724200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
575,2188,140730500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1047,2725,140707580,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1102,2791,141409960,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1103,2792,142439100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1312,300238,140825950,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1515,300484,150039240,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1536,300506,145228450,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
# trade = pd.read_table(r'J:\LEVEL2_shenzhen\2016\03\29\SZL2_TRADE_20160329\SZL2_TRADE_20160329_AM.txt', header=None)
trade.columns = ["date", "e", "a", "b",'c', 'd', 'id', 'f', 'g', 'h','i','k']
pd.set_option("max_rows", 200)
trade[(trade['id'] == 36) & (trade['e'] > 140300000)].head(100)

,date,e,a,b,c,d,id,f,g,h,i,k


In [104]:
pd.set_option("max_columns", 200)
snapshot[(snapshot["StockID"]==300238) & (snapshot["time"] > 140300000)].head(40)

,Date,SENDTIME,StockID,DATATIMESTAMP,PreClosePx,openPrice,HighPx,LowPx,close,NUMTRADES,cum_volume,cum_amount,TOTALLONGPOSITION,PERATIO1,PERATIO2,ENDOFDAYMAKER,TOTALOFFERQTY,WEIGHTEDAVGOFFERPX,TOTALBIDQTY,WEIGHTEDAVGBIDPX,ask10p,ask9p,ask8p,ask7p,ask6p,ask5p,ask4p,ask3p,ask2p,ask1p,bid1p,bid2p,bid3p,bid4p,bid5p,bid6p,bid7p,bid8p,bid9p,bid10p,ask10q,ask9q,ask8q,ask7q,ask6q,ask5q,ask4q,ask3q,ask2q,ask1q,bid1q,bid2q,bid3q,bid4q,bid5q,bid6q,bid7q,bid8q,bid9q,bid10q,NUMORDERS_B1,NOORDERS _B1,ORDERQTY_B1,NUMORDERS_S1,NOORDERS _S1,ORDERQTY_S1,time,OrigTime,cum_max,group,time_diff,volume_update,datetime,time_interval,time1,time2,time3,time4,tn_update,99%,95%,90%
6627655,20160329,140308,300238,140302790,56.67,51.0,51.0,51.0,51.0,267,173900,8868900.0,0,276.44,0.0,N,26412818,51.38,0,0.0,51.59,51.51,51.5,51.4,51.1,51.05,51.04,51.02,51.01,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24251949,40800,31800,200,1500,300,200,171309,500,3000,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2348,50,257795|13000|13000|18663|555200|20200|22700|34...,140302790,20160329140302790,180600,4676,60030.0,0.0,2016-03-29 14:03:02,20.0,93001560,130004140,150039250.0,140825950,0.0,7.35,2.0,1.0
6627656,20160329,140311,300238,140304540,56.67,51.0,51.0,51.0,51.0,267,173900,8868900.0,0,276.44,0.0,N,26412918,51.38,0,0.0,51.59,51.51,51.5,51.4,51.1,51.05,51.04,51.02,51.01,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24252049,40800,31800,200,1500,300,200,171309,500,3000,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2349,50,257795|13000|13000|18663|555200|20200|22700|34...,140304540,20160329140304540,180600,4676,1750.0,0.0,2016-03-29 14:03:04,2.0,93001560,130004140,150039250.0,140825950,0.0,7.35,2.0,1.0
6627657,20160329,140314,300238,140304540,56.67,51.0,51.0,51.0,51.0,267,173900,8868900.0,0,276.44,0.0,N,26412918,51.38,0,0.0,51.59,51.51,51.5,51.4,51.1,51.05,51.04,51.02,51.01,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24252049,40800,31800,200,1500,300,200,171309,500,3000,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2349,50,257795|13000|13000|18663|555200|20200|22700|34...,140304540,20160329140304540,180600,4676,0.0,0.0,2016-03-29 14:03:04,0.0,93001560,130004140,150039250.0,140825950,0.0,7.35,2.0,1.0
6627658,20160329,140317,300238,140304540,56.67,51.0,51.0,51.0,51.0,267,173900,8868900.0,0,276.44,0.0,N,26412918,51.38,0,0.0,51.59,51.51,51.5,51.4,51.1,51.05,51.04,51.02,51.01,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24252049,40800,31800,200,1500,300,200,171309,500,3000,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2349,50,257795|13000|13000|18663|555200|20200|22700|34...,140304540,20160329140304540,180600,4676,0.0,0.0,2016-03-29 14:03:04,0.0,93001560,130004140,150039250.0,140825950,0.0,7.35,2.0,1.0
6627659,20160329,140320,300238,140304540,56.67,51.0,51.0,51.0,51.0,267,173900,8868900.0,0,276.44,0.0,N,26412918,51.38,0,0.0,51.59,51.51,51.5,51.4,51.1,51.05,51.04,51.02,51.01,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24252049,40800,31800,200,1500,300,200,171309,500,3000,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2349,50,257795|13000|13000|18663|555200|20200|22700|34...,140304540,20160329140304540,180600,4676,0.0,0.0,2016-03-29 14:03:04,0.0,93001560,130004140,150039250.0,140825950,0.0,7.35,2.0,1.0
6627660,20160329,140323,300238,140304540,56.67,51.0,51.0,51.0,51.0,267,173900,8868900.0,0,276.44,0.0,N,26412918,51.38,0,0.0,51.59,51.51,51.5,51.4,51.1,51.05,51.04,51.02,51.01,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24252049,40800,31800,200,1500,300,200,171309,500,3000,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2349,50,257795|13000|13000|18663|555200|20200|22700|34...,140304540,20160329140304540,180600,4676,0.0,0.0,2016-03-29 14:03:04,0.0,93001560,130004140,150039250.0,140825950,0.0,7.35,2.0,1.0
6627661,20160329,140326,300238,140304540,56.67,51.0,51.0,51.0,51.0,267,173900,8868900.0,0,276.44,0.0,N,26412918,51.38,0,0.0,51.59,51.51,51.5,51.4,51.1,51.05,51.04,51.02,51.01,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24252049,40800,31800,200,1500,300,200,171309,500,3000,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2349,50,257795|13000|13000|18663|555200|20200|22700|34...,140304540,20160329140304540,18

In [ ]:
re = snapshot[(snapshot["StockID"] == secid) & (snapshot["time_interval"] >= 60) & (snapshot["volume_update"] > snapshot[(snapshot["StockID"] == secid) & (snapshot["time"] > 93000000) & (snapshot["time"] < 145700000) & (snapshot["time"] != snapshot["time2"])]["volume_update"].describe([0.99])["99%"]) \
        & (snapshot["time"] > snapshot["time1"]) & (snapshot["time"] != snapshot["time2"]) & (snapshot["time"] != snapshot["time3"])]

In [8]:
l1 = []
l2 = []
name = []
mr = pd.DataFrame()
for secid in np.sort(snapshot["StockID"].unique()):
    re = snapshot[(snapshot["StockID"] == secid) & (snapshot["time_interval"] >= 60) & (snapshot["volume_update"] > snapshot[(snapshot["StockID"] == secid) & (snapshot["time"] > 93000000) & (snapshot["time"] < 145700000) & (snapshot["time"] != snapshot["time2"])]["volume_update"].describe([0.99])["99%"]) \
        & (snapshot["time"] > snapshot["time1"]) & (snapshot["time"] != snapshot["time2"]) & (snapshot["time"] != snapshot["time3"])]
    l1.append(re.shape[0])
    name.append(secid)
    if re.shape[0] == 1:
        l2.append(re["time"].values[0])
    else:
        l2.append(re["time"].values)
mr["StockID"] = name
mr["count"] = l1
mr["time"] = l2
mr

,StockID,count,time
0,1,1,140735860
1,5,1,140735370
2,6,1,140735090
3,8,1,140735800
4,9,1,140737260
...,...,...,...
1532,300501,1,140735040
1533,300502,1,140735720
1534,300503,1,140735670
1535,300505,1,140736980


In [24]:
mr[mr["count"] != 1]

,StockID,count,time
22,32,0,[]
24,36,0,[]
40,61,0,[]
44,68,0,[]
46,70,0,[]
...,...,...,...
1517,300486,0,[]
1519,300488,0,[]
1530,300499,0,[]
1531,300500,0,[]


In [33]:
p90[p90["StockID"] == 36]

,StockID,volume_update
24,36,8490.0


In [167]:
pd.set_option("max_columns", 200)
snapshot[(snapshot["StockID"]==967) & (snapshot["time"] > 140410000)].head(100)

,Date,SENDTIME,StockID,DATATIMESTAMP,PreClosePx,openPrice,HighPx,LowPx,close,NUMTRADES,cum_volume,cum_amount,TOTALLONGPOSITION,PERATIO1,PERATIO2,ENDOFDAYMAKER,TOTALOFFERQTY,WEIGHTEDAVGOFFERPX,TOTALBIDQTY,WEIGHTEDAVGBIDPX,ask10p,ask9p,ask8p,ask7p,ask6p,ask5p,ask4p,ask3p,ask2p,ask1p,bid1p,bid2p,bid3p,bid4p,bid5p,bid6p,bid7p,bid8p,bid9p,bid10p,ask10q,ask9q,ask8q,ask7q,ask6q,ask5q,ask4q,ask3q,ask2q,ask1q,bid1q,bid2q,bid3q,bid4q,bid5q,bid6q,bid7q,bid8q,bid9q,bid10q,NUMORDERS_B1,NOORDERS _B1,ORDERQTY_B1,NUMORDERS_S1,NOORDERS _S1,ORDERQTY_S1,time,OrigTime,cum_max,group,trade_diff,time_diff,volume_update,time1,time2,time3,time4,datetime,time_interval
125080,20160329,140740,36,140734100,8.16,8.2,8.23,7.87,7.99,5267,13394885,1.073817e+08,0,266.67,0.0,N,3832783,8.50,2708215,7.76,8.08,8.07,8.06,8.05,8.04,8.03,8.02,8.01,8.00,7.99,7.98,7.97,7.96,7.95,7.94,7.93,7.92,7.91,7.90,7.89,17400,33200,13300,24500,33600,64300,51400,1500,24600,41800,19900,23300,52500,116100,11400,16200,51915,54000,228000,67300,16,16,300|3000|1000|100|1000|400|100|1000|2000|2000|...,3,3,4800|10100|2500,140734100,20160329140734100,17463971,4691,20.0,331400.0,25000.0,93000320,130001600,150019780.0,140734100,2016-03-29 14:07:34,212.0
125081,20160329,140743,36,140739880,8.16,8.2,8.23,7.87,7.99,5267,13394885,1.073817e+08,0,266.67,0.0,N,3832783,8.50,2707815,7.76,8.08,8.07,8.06,8.05,8.04,8.03,8.02,8.01,8.00,7.99,7.98,7.97,7.96,7.95,7.94,7.93,7.92,7.91,7.90,7.89,17400,33200,13300,24500,33600,64300,51400,1500,24600,41800,19900,23300,52500,115700,11400,16200,51915,54000,228000,67300,16,16,300|3000|1000|100|1000|400|100|1000|2000|2000|...,3,3,4800|10100|2500,140739880,20160329140739880,17463971,4691,0.0,5780.0,0.0,93000320,130001600,150019780.0,140734100,2016-03-29 14:07:39,5.0
125082,20160329,140746,36,140739880,8.16,8.2,8.23,7.87,7.99,5267,13394885,1.073817e+08,0,266.67,0.0,N,3832783,8.50,2707815,7.76,8.08,8.07,8.06,8.05,8.04,8.03,8.02,8.01,8.00,7.99,7.98,7.97,7.96,7.95,7.94,7.93,7.92,7.91,7.90,7.89,17400,33200,13300,24500,33600,64300,51400,1500,24600,41800,19900,23300,52500,115700,11400,16200,51915,54000,228000,67300,16,16,300|3000|1000|100|1000|400|100|1000|2000|2000|...,3,3,4800|10100|2500,140739880,20160329140739880,17463971,4691,0.0,0.0,0.0,93000320,130001600,150019780.0,140734100,2016-03-29 14:07:39,0.0
125083,20160329,140749,36,140746380,8.16,8.2,8.23,7.87,7.99,5273,13400085,1.074232e+08,0,266.67,0.0,N,3833483,8.50,2703415,7.76,8.08,8.07,8.06,8.05,8.04,8.03,8.02,8.01,8.00,7.99,7.98,7.97,7.96,7.95,7.94,7.93,7.92,7.91,7.90,7.89,17200,33200,13300,24500,33600,64300,51400,1500,24600,41800,15400,23300,51500,116700,11400,16200,51915,54000,228100,67300,13,13,400|400|100|1000|2000|2000|500|1000|3000|1000|...,3,3,4600|10100|2500,140746380,20160329140746380,17463971,4691,6.0,6500.0,5200.0,93000320,130001600,150019780.0,140734100,2016-03-29 14:07:46,7.0
125084,20160329,140752,36,140746560,8.16,8.2,8.23,7.87,7.98,5278,13402885,1.074455e+08,0,266.67,0.0,N,3833483,8.50,2700615,7.76,8.08,8.07,8.06,8.05,8.04,8.03,8.02,8.01,8.00,7.99,7.98,7.97,7.96,7.95,7.94,7.93,7.92,7.91,7.90,7.89,17200,33200,13300,24500,33600,64300,51400,1500,24600,41800,12600,23300,51500,116700,11400,16200,51915,54000,228100,67300,9,9,1100|2000|500|1000|3000|1000|3000|500|500,3,3,4600|10100|2500,140746560,20160329140746560,17463971,4691,5.0,180.0,2800.0,93000320,130001600,150019780.0,140734100,2016-03-29 14:07:46,0.0
125085,20160329,140755,36,140751840,8.16,8.2,8.23,7.87,7.98,5278,13402885,1.074455e+08,0,266.67,0.0,N,3833483,8.50,2700015,7.76,8.08,8.07,8.06,8.05,8.04,8.03,8.02,8.01,8.00,7.99,7.98,7.97,7.96,7.95,7.94,7.93,7.92,7.91,7.90,7.89,17200,33200,13300,24500,33600,64300,51400,1500,24600,41800,12600,23300,51500,116100,11400,16200,51915,54000,228100,67300,9,9,1100|2000|500|1000|3000|1000|3000|500|500,3,3,4600|10100|2500,140751840,20160329140751840,17463971,4691,0.0,5280.0,0.0,93000320,130001600,150019780.0,140734100,2016-03-29 14:07:51,5.0
125086,20160329,140758,36,140753490,8.16,8.2,8.23,7.87,7.98,5278,13402885,1.

In [141]:
snapshot[snapshot["time"] < 140414000]["time"].max()

140413820

#### Condition 1: timeInv >= 80sec & volumeThisUpdate > 90 percentile of volumeThisUpdate

In [357]:
l1 = []
l2 = []
name = []
mr1 = pd.DataFrame()

for secid in np.sort(snapshot["StockID"].unique()):
    re = snapshot[(snapshot["StockID"] == secid) & (snapshot["time_interval"] > 35) & (snapshot["volume_update"] > snapshot[(snapshot["StockID"] == secid) & (snapshot["time"] > 93000000) & (snapshot["time"] < 145700000) & (snapshot["time"] != snapshot["time2"])]["volume_update"].describe([0.99])["99%"]) \
        & (snapshot["time"] > snapshot["time1"]) & (snapshot["time"] != snapshot["time2"]) & (snapshot["time"] != snapshot["time3"])]
    l1.append(re.shape[0])
    name.append(secid)
    if re.shape[0] == 1:
        l2.append(re["time"].values[0])
    else:
        print(re)
        l2.append(re["time"].values)
mr1["StockID"] = name
mr1["count"] = l1
mr1["time"] = l2
mr1

Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, SecurityIDsource, MDStreamID, PreClosePx, PxChnage1, PXChange2, openPrice, HighPx, LowPx, close, NumTrades, cum_volume, cum_amount, PE1, PE2, TradingPhase, totalofferqty, wa_offerPrice, totalbidqty, wa_bidPrice, PreNAV, RealTimeNAV, WarrantPremiumRate, UpLimitPx, DownLimitPx, TotalLongPosition, time, SendTime_y, ercvtime_y, dbtime_y, ask1p, bid1p, ask1q, bid1q, ask2p, bid2p, ask2q, bid2q, ask3p, bid3p, ask3q, bid3q, ask4p, bid4p, ask4q, bid4q, ask5p, bid5p, ask5q, bid5q, ask6p, bid6p, ask6q, bid6q, ask7p, bid7p, ask7q, bid7q, ask8p, bid8p, ask8q, bid8q, ask9p, bid9p, ask9q, bid9q, ask10p, bid10p, ask10q, bid10q, NUMORDERS_B1, NOORDERS_B1, ORDERQTY_B1, NUMORDERS_S1, NOORDERS_S1, ORDERQTY_S1, cum_max, group, time_diff, volume_update, datetime, time_interval, time1, time2, time3, time4]
Index: []
Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, Sec

Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, SecurityIDsource, MDStreamID, PreClosePx, PxChnage1, PXChange2, openPrice, HighPx, LowPx, close, NumTrades, cum_volume, cum_amount, PE1, PE2, TradingPhase, totalofferqty, wa_offerPrice, totalbidqty, wa_bidPrice, PreNAV, RealTimeNAV, WarrantPremiumRate, UpLimitPx, DownLimitPx, TotalLongPosition, time, SendTime_y, ercvtime_y, dbtime_y, ask1p, bid1p, ask1q, bid1q, ask2p, bid2p, ask2q, bid2q, ask3p, bid3p, ask3q, bid3q, ask4p, bid4p, ask4q, bid4q, ask5p, bid5p, ask5q, bid5q, ask6p, bid6p, ask6q, bid6q, ask7p, bid7p, ask7q, bid7q, ask8p, bid8p, ask8q, bid8q, ask9p, bid9p, ask9q, bid9q, ask10p, bid10p, ask10q, bid10q, NUMORDERS_B1, NOORDERS_B1, ORDERQTY_B1, NUMORDERS_S1, NOORDERS_S1, ORDERQTY_S1, cum_max, group, time_diff, volume_update, datetime, time_interval, time1, time2, time3, time4]
Index: []
Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, Sec

Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, SecurityIDsource, MDStreamID, PreClosePx, PxChnage1, PXChange2, openPrice, HighPx, LowPx, close, NumTrades, cum_volume, cum_amount, PE1, PE2, TradingPhase, totalofferqty, wa_offerPrice, totalbidqty, wa_bidPrice, PreNAV, RealTimeNAV, WarrantPremiumRate, UpLimitPx, DownLimitPx, TotalLongPosition, time, SendTime_y, ercvtime_y, dbtime_y, ask1p, bid1p, ask1q, bid1q, ask2p, bid2p, ask2q, bid2q, ask3p, bid3p, ask3q, bid3q, ask4p, bid4p, ask4q, bid4q, ask5p, bid5p, ask5q, bid5q, ask6p, bid6p, ask6q, bid6q, ask7p, bid7p, ask7q, bid7q, ask8p, bid8p, ask8q, bid8q, ask9p, bid9p, ask9q, bid9q, ask10p, bid10p, ask10q, bid10q, NUMORDERS_B1, NOORDERS_B1, ORDERQTY_B1, NUMORDERS_S1, NOORDERS_S1, ORDERQTY_S1, cum_max, group, time_diff, volume_update, datetime, time_interval, time1, time2, time3, time4]
Index: []
Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, Sec

Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, SecurityIDsource, MDStreamID, PreClosePx, PxChnage1, PXChange2, openPrice, HighPx, LowPx, close, NumTrades, cum_volume, cum_amount, PE1, PE2, TradingPhase, totalofferqty, wa_offerPrice, totalbidqty, wa_bidPrice, PreNAV, RealTimeNAV, WarrantPremiumRate, UpLimitPx, DownLimitPx, TotalLongPosition, time, SendTime_y, ercvtime_y, dbtime_y, ask1p, bid1p, ask1q, bid1q, ask2p, bid2p, ask2q, bid2q, ask3p, bid3p, ask3q, bid3q, ask4p, bid4p, ask4q, bid4q, ask5p, bid5p, ask5q, bid5q, ask6p, bid6p, ask6q, bid6q, ask7p, bid7p, ask7q, bid7q, ask8p, bid8p, ask8q, bid8q, ask9p, bid9p, ask9q, bid9q, ask10p, bid10p, ask10q, bid10q, NUMORDERS_B1, NOORDERS_B1, ORDERQTY_B1, NUMORDERS_S1, NOORDERS_S1, ORDERQTY_S1, cum_max, group, time_diff, volume_update, datetime, time_interval, time1, time2, time3, time4]
Index: []
Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, Sec

Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, SecurityIDsource, MDStreamID, PreClosePx, PxChnage1, PXChange2, openPrice, HighPx, LowPx, close, NumTrades, cum_volume, cum_amount, PE1, PE2, TradingPhase, totalofferqty, wa_offerPrice, totalbidqty, wa_bidPrice, PreNAV, RealTimeNAV, WarrantPremiumRate, UpLimitPx, DownLimitPx, TotalLongPosition, time, SendTime_y, ercvtime_y, dbtime_y, ask1p, bid1p, ask1q, bid1q, ask2p, bid2p, ask2q, bid2q, ask3p, bid3p, ask3q, bid3q, ask4p, bid4p, ask4q, bid4q, ask5p, bid5p, ask5q, bid5q, ask6p, bid6p, ask6q, bid6q, ask7p, bid7p, ask7q, bid7q, ask8p, bid8p, ask8q, bid8q, ask9p, bid9p, ask9q, bid9q, ask10p, bid10p, ask10q, bid10q, NUMORDERS_B1, NOORDERS_B1, ORDERQTY_B1, NUMORDERS_S1, NOORDERS_S1, ORDERQTY_S1, cum_max, group, time_diff, volume_update, datetime, time_interval, time1, time2, time3, time4]
Index: []
Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, Sec

...

Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, SecurityIDsource, MDStreamID, PreClosePx, PxChnage1, PXChange2, openPrice, HighPx, LowPx, close, NumTrades, cum_volume, cum_amount, PE1, PE2, TradingPhase, totalofferqty, wa_offerPrice, totalbidqty, wa_bidPrice, PreNAV, RealTimeNAV, WarrantPremiumRate, UpLimitPx, DownLimitPx, TotalLongPosition, time, SendTime_y, ercvtime_y, dbtime_y, ask1p, bid1p, ask1q, bid1q, ask2p, bid2p, ask2q, bid2q, ask3p, bid3p, ask3q, bid3q, ask4p, bid4p, ask4q, bid4q, ask5p, bid5p, ask5q, bid5q, ask6p, bid6p, ask6q, bid6q, ask7p, bid7p, ask7q, bid7q, ask8p, bid8p, ask8q, bid8q, ask9p, bid9p, ask9q, bid9q, ask10p, bid10p, ask10q, bid10q, NUMORDERS_B1, NOORDERS_B1, ORDERQTY_B1, NUMORDERS_S1, NOORDERS_S1, ORDERQTY_S1, cum_max, group, time_diff, volume_update, datetime, time_interval, time1, time2, time3, time4]
Index: []
Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, Sec

Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, SecurityIDsource, MDStreamID, PreClosePx, PxChnage1, PXChange2, openPrice, HighPx, LowPx, close, NumTrades, cum_volume, cum_amount, PE1, PE2, TradingPhase, totalofferqty, wa_offerPrice, totalbidqty, wa_bidPrice, PreNAV, RealTimeNAV, WarrantPremiumRate, UpLimitPx, DownLimitPx, TotalLongPosition, time, SendTime_y, ercvtime_y, dbtime_y, ask1p, bid1p, ask1q, bid1q, ask2p, bid2p, ask2q, bid2q, ask3p, bid3p, ask3q, bid3q, ask4p, bid4p, ask4q, bid4q, ask5p, bid5p, ask5q, bid5q, ask6p, bid6p, ask6q, bid6q, ask7p, bid7p, ask7q, bid7q, ask8p, bid8p, ask8q, bid8q, ask9p, bid9p, ask9q, bid9q, ask10p, bid10p, ask10q, bid10q, NUMORDERS_B1, NOORDERS_B1, ORDERQTY_B1, NUMORDERS_S1, NOORDERS_S1, ORDERQTY_S1, cum_max, group, time_diff, volume_update, datetime, time_interval, time1, time2, time3, time4]
Index: []
Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, Sec

Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, SecurityIDsource, MDStreamID, PreClosePx, PxChnage1, PXChange2, openPrice, HighPx, LowPx, close, NumTrades, cum_volume, cum_amount, PE1, PE2, TradingPhase, totalofferqty, wa_offerPrice, totalbidqty, wa_bidPrice, PreNAV, RealTimeNAV, WarrantPremiumRate, UpLimitPx, DownLimitPx, TotalLongPosition, time, SendTime_y, ercvtime_y, dbtime_y, ask1p, bid1p, ask1q, bid1q, ask2p, bid2p, ask2q, bid2q, ask3p, bid3p, ask3q, bid3q, ask4p, bid4p, ask4q, bid4q, ask5p, bid5p, ask5q, bid5q, ask6p, bid6p, ask6q, bid6q, ask7p, bid7p, ask7q, bid7q, ask8p, bid8p, ask8q, bid8q, ask9p, bid9p, ask9q, bid9q, ask10p, bid10p, ask10q, bid10q, NUMORDERS_B1, NOORDERS_B1, ORDERQTY_B1, NUMORDERS_S1, NOORDERS_S1, ORDERQTY_S1, cum_max, group, time_diff, volume_update, datetime, time_interval, time1, time2, time3, time4]
Index: []
Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, Sec

Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, SecurityIDsource, MDStreamID, PreClosePx, PxChnage1, PXChange2, openPrice, HighPx, LowPx, close, NumTrades, cum_volume, cum_amount, PE1, PE2, TradingPhase, totalofferqty, wa_offerPrice, totalbidqty, wa_bidPrice, PreNAV, RealTimeNAV, WarrantPremiumRate, UpLimitPx, DownLimitPx, TotalLongPosition, time, SendTime_y, ercvtime_y, dbtime_y, ask1p, bid1p, ask1q, bid1q, ask2p, bid2p, ask2q, bid2q, ask3p, bid3p, ask3q, bid3q, ask4p, bid4p, ask4q, bid4q, ask5p, bid5p, ask5q, bid5q, ask6p, bid6p, ask6q, bid6q, ask7p, bid7p, ask7q, bid7q, ask8p, bid8p, ask8q, bid8q, ask9p, bid9p, ask9q, bid9q, ask10p, bid10p, ask10q, bid10q, NUMORDERS_B1, NOORDERS_B1, ORDERQTY_B1, NUMORDERS_S1, NOORDERS_S1, ORDERQTY_S1, cum_max, group, time_diff, volume_update, datetime, time_interval, time1, time2, time3, time4]
Index: []
Empty DataFrame
Columns: [Date, OrigTime, SendTime_x, ercvtime_x, dbtime_x, ChannelNo, StockID, Sec

,StockID,count,time
0,1,1,105236000
1,2,1,105236000
2,4,0,[]
3,5,0,[]
4,6,0,[]
...,...,...,...
1809,300550,0,[]
1810,300551,0,[]
1811,300552,0,[]
1812,300553,0,[]


In [356]:
mr1[(mr1["count"] != 1) & (mr1["count"] != 0)].shape[0]

54

In [359]:
mr1[mr1["count"] != 1]

,StockID,count,time
2,4,0,[]
3,5,0,[]
4,6,0,[]
6,8,0,[]
7,9,0,[]
...,...,...,...
1809,300550,0,[]
1810,300551,0,[]
1811,300552,0,[]
1812,300553,0,[]


In [350]:
secid = 48
snapshot[(snapshot["StockID"] == secid) & (snapshot["time_interval"] > 35) & (snapshot["volume_update"] > snapshot[(snapshot["StockID"] == secid) & (snapshot["time"] > 93000000) & (snapshot["time"] < 145700000) & (snapshot["time"] != snapshot["time2"])]["volume_update"].describe([0.95])["95%"]) \
        & (snapshot["time"] > snapshot["time1"]) & (snapshot["time"] != snapshot["time2"]) & (snapshot["time"] != snapshot["time3"])]

,Date,OrigTime,SendTime_x,ercvtime_x,dbtime_x,ChannelNo,StockID,SecurityIDsource,MDStreamID,PreClosePx,PxChnage1,PXChange2,openPrice,HighPx,LowPx,close,NumTrades,cum_volume,cum_amount,PE1,PE2,TradingPhase,totalofferqty,wa_offerPrice,totalbidqty,wa_bidPrice,PreNAV,RealTimeNAV,WarrantPremiumRate,UpLimitPx,DownLimitPx,TotalLongPosition,time,SendTime_y,ercvtime_y,dbtime_y,ask1p,bid1p,ask1q,bid1q,ask2p,bid2p,ask2q,bid2q,ask3p,bid3p,ask3q,bid3q,ask4p,bid4p,ask4q,bid4q,ask5p,bid5p,ask5q,bid5q,ask6p,bid6p,ask6q,bid6q,ask7p,bid7p,ask7q,bid7q,ask8p,bid8p,ask8q,bid8q,ask9p,bid9p,ask9q,bid9q,ask10p,bid10p,ask10q,bid10q,NUMORDERS_B1,NOORDERS_B1,ORDERQTY_B1,NUMORDERS_S1,NOORDERS_S1,ORDERQTY_S1,cum_max,group,time_diff,volume_update,datetime,time_interval,time1,time2,time3,time4
118521,20161031,20161031102021000,20161103145156523,20161103145158519,20161103145200878,1011,48,102,10,34.73,-0.04,0.04,34.75,35.0,34.51,34.69,218,115241,4004672.55,66.46,0.0,T0,325194.0,35.51,87759.0,34.26,0.0,0.0,0.0,38.2,31.26,0.0,102021000,20161103145156523,20161103145158519,20161103145200878,34.69,34.65,300,3000,34.70,34.64,22200,300,34.71,34.62,5200,1000,34.77,34.61,300,2000,34.78,34.60,1500,3800,34.80,34.55,900,4759,34.81,34.53,900,1500,34.82,34.52,5200,10500,34.84,34.50,100,5200,34.85,34.42,2400,500,1,1,3000,1,1,300,455625,3400,79000.0,4000.0,2016-10-31 10:20:21,39.0,95654000,130003000.0,150003000.0,105242000.0
118686,20161031,20161031105242000,20161103145745867,20161103145746477,20161103145747622,1011,48,102,10,34.73,0.08,0.00,34.75,35.0,34.51,34.81,300,183793,6385793.35,66.69,0.0,T0,343045.0,35.58,108559.0,34.32,0.0,0.0,0.0,38.2,31.26,0.0,105242000,20161103145745867,20161103145746477,20161103145747622,34.81,34.73,700,1100,34.82,34.72,6400,2500,34.84,34.69,100,1700,34.85,34.68,2400,500,34.86,34.65,2200,200,34.87,34.63,4500,100,34.88,34.62,24000,1000,34.90,34.61,15100,3700,34.92,34.60,400,4600,34.95,34.55,200,4759,1,1,1100,1,1,700,455625,3508,112000.0,2300.0,2016-10-31 10:52:42,72.0,95654000,130003000.0,150003000.0,105242000.0
119022,20161031,20161031134442000,20161103151212812,20161103151213089,20161103151213760,1011,48,102,10,34.73,-0.04,0.00,34.75,35.0,34.51,34.69,450,317193,11012696.35,66.46,0.0,T0,375546.0,35.62,134159.0,34.33,0.0,0.0,0.0,38.2,31.26,0.0,134442000,20161103151212812,20161103151213089,20161103151213760,34.70,34.69,1800,100,34.71,34.62,4600,17700,34.74,34.61,3700,600,34.75,34.60,2600,800,34.78,34.59,1100,1500,34.80,34.58,5801,100,34.81,34.56,9100,5000,34.82,34.55,3000,5459,34.84,34.53,100,1600,34.85,34.52,2100,15100,1,1,100,1,1,1800,455625,4481,42000.0,2900.0,2016-10-31 13:44:42,42.0,95654000,130003000.0,150003000.0,105242000.0
119024,20161031,20161031134551000,20161103151215928,20161103151215964,20161103151216030,1011,48,102,10,34.73,-0.04,0.00,34.75,35.0,34.51,34.69,452,322193,11186146.35,66.46,0.0,T0,375546.0,35.62,134159.0,34.33,0.0,0.0,0.0,38.2,31.26,0.0,134551000,20161103151215928,20161103151215964,20161103151216030,34.70,34.69,1800,100,34.71,34.62,4600,17700,34.74,34.61,3700,600,34.75,34.60,2600,800,34.78,34.59,1100,1500,34.80,34.58,5801,100,34.81,34.56,9100,5000,34.82,34.55,3000,5459,34.84,34.53,100,1600,34.85,34.52,2100,15100,1,1,100,1,1,1800,455625,4485,45000.0,5000.0,2016-10-31 13:45:51,45.0,95654000,130003000.0,150003000.0,105242000.0


In [329]:
snapshot[(snapshot["StockID"] == secid) & (snapshot["time"] > 93000000) & (snapshot["time"] < 145700000)]["volume_update"].describe([0.95])["95%"]

3900.0

In [344]:
pd.set_option("max_columns", 200)
snapshot[(snapshot["StockID"]==23) & (snapshot["time"] > 105000000)].head(100)

,Date,OrigTime,SendTime_x,ercvtime_x,dbtime_x,ChannelNo,StockID,SecurityIDsource,MDStreamID,PreClosePx,PxChnage1,PXChange2,openPrice,HighPx,LowPx,close,NumTrades,cum_volume,cum_amount,PE1,PE2,TradingPhase,totalofferqty,wa_offerPrice,totalbidqty,wa_bidPrice,PreNAV,RealTimeNAV,WarrantPremiumRate,UpLimitPx,DownLimitPx,TotalLongPosition,time,SendTime_y,ercvtime_y,dbtime_y,ask1p,bid1p,ask1q,bid1q,ask2p,bid2p,ask2q,bid2q,ask3p,bid3p,ask3q,bid3q,ask4p,bid4p,ask4q,bid4q,ask5p,bid5p,ask5q,bid5q,ask6p,bid6p,ask6q,bid6q,ask7p,bid7p,ask7q,bid7q,ask8p,bid8p,ask8q,bid8q,ask9p,bid9p,ask9q,bid9q,ask10p,bid10p,ask10q,bid10q,NUMORDERS_B1,NOORDERS_B1,ORDERQTY_B1,NUMORDERS_S1,NOORDERS_S1,ORDERQTY_S1,cum_max,group,time_diff,volume_update,datetime,time_interval,time1,time2,time3,time4
65409,20161031,20161031105045000,20161103145743134,20161103145743625,20161103145744882,1011,23,102,10,30.88,0.25,-0.01,30.88,31.36,30.7,31.13,648,671155,20808754.11,161.63,0.0,T0,354950.0,32.20,174700.0,30.69,0.0,0.0,0.0,33.97,27.79,0.0,105045000,20161103145743134,20161103145743625,20161103145744882,31.12,31.03,500,1200,31.16,31.01,2100,200,31.20,31.00,6800,200,31.21,30.98,14900,4700,31.24,30.96,500,500,31.25,30.95,7600,2300,31.26,30.93,100,2000,31.27,30.92,1500,1900,31.28,30.90,22700,1400,31.29,30.89,27000,3500,2,2,1100|100,1,1,500,1809661,3501,88000.0,0.0,2016-10-31 10:50:45,48.0,95648000,130000000.0,150003000.0,105315000.0
65410,20161031,20161031105051000,20161103145743186,20161103145743697,20161103145744984,1011,23,102,10,30.88,0.25,-0.01,30.88,31.36,30.7,31.13,648,671155,20808754.11,161.63,0.0,T0,352050.0,32.21,174700.0,30.69,0.0,0.0,0.0,33.97,27.79,0.0,105051000,20161103145743186,20161103145743697,20161103145744984,31.12,31.03,500,1200,31.16,31.01,2100,200,31.20,31.00,3900,200,31.21,30.98,14900,4700,31.24,30.96,500,500,31.25,30.95,7600,2300,31.26,30.93,100,2000,31.27,30.92,1500,1900,31.28,30.90,22700,1400,31.29,30.89,27000,3500,2,2,1100|100,1,1,500,1809661,3501,6000.0,0.0,2016-10-31 10:50:51,6.0,95648000,130000000.0,150003000.0,105315000.0
65411,20161031,20161031105109000,20161103145743374,20161103145744022,20161103145745385,1011,23,102,10,30.88,0.25,-0.01,30.88,31.36,30.7,31.13,648,671155,20808754.11,161.63,0.0,T0,351550.0,32.21,174700.0,30.69,0.0,0.0,0.0,33.97,27.79,0.0,105109000,20161103145743374,20161103145744022,20161103145745385,31.16,31.03,2100,1200,31.20,31.01,3900,200,31.21,31.00,14900,200,31.24,30.98,500,4700,31.25,30.96,7600,500,31.26,30.95,100,2300,31.27,30.93,1500,2000,31.28,30.92,22700,1900,31.29,30.90,27000,1400,31.30,30.89,18300,3500,2,2,1100|100,1,1,2100,1809661,3503,58000.0,0.0,2016-10-31 10:51:09,18.0,95648000,130000000.0,150003000.0,105315000.0
65412,20161031,20161031105115000,20161103145745400,20161103145745468,20161103145745621,1011,23,102,10,30.88,0.25,-0.01,30.88,31.36,30.7,31.13,648,671155,20808754.11,161.63,0.0,T0,351550.0,32.21,175200.0,30.69,0.0,0.0,0.0,33.97,27.79,0.0,105115000,20161103145745400,20161103145745468,20161103145745621,31.16,31.03,2100,1200,31.20,31.01,3900,200,31.21,31.00,14900,700,31.24,30.98,500,4700,31.25,30.96,7600,500,31.26,30.95,100,2300,31.27,30.93,1500,2000,31.28,30.92,22700,1900,31.29,30.90,27000,1400,31.30,30.89,18300,3500,2,2,1100|100,1,1,2100,1809661,3503,6000.0,0.0,2016-10-31 10:51:15,6.0,95648000,130000000.0,150003000.0,105315000.0
65413,20161031,20161031105118000,20161103145745426,20161103145745536,20161103145745762,1011,23,102,10,30.88,0.25,-0.01,30.88,31.36,30.7,31.13,648,671155,20808754.11,161.63,0.0,T0,352550.0,32.21,175200.0,30.69,0.0,0.0,0.0,33.97,27.79,0.0,105118000,20161103145745426,20161103145745536,20161103145745762,31.16,31.03,2100,1200,31.20,31.01,4900,200,31.21,31.00,14900,700,31.24,30.98,500,4700,31.25,30.96,7600,500,31.26,30.95,100,2300,31.27,30.93,1500,2000,31.28,30.92,22700,1900,31.29,30.90,27000,1400,31.30,30.89,18300,3500,2,2,1100|100,1,1,2100,1809661,3503,3000.0,0.0,2016-10-31 10:51:18,3.0,95648000,130000000.0,150003000.0,105315000.0
65414,20161031,20161031105121000,20161103145745449,20

In [262]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)

df = pd.read_table(r"J:\LEVEL2_shenzhen\2016\03\29\SZL2_TRADE_20160329\SZL2_TRADE_20160329_AM.txt", header=None)
df.columns=["TRADEDATE", "SENDTIME", "SETNO", "RECNO", "BUYORDERRECNO", "SELLORDERRECNO", "SECURITYID", "PRICE",
           "TRADEQTY", "ORDERKIND", "FUNCTIONCODE", "TRADETIME"]

#### Condition 2: timeInv >= 60sec & (abs(bid1p/bid1p_L1) > 30bps or abs(ask1p/ask1p_L1) > 30bps)

In [184]:
snapshot["rate1"] = snapshot.groupby("StockID")["bid1p"].apply(lambda x: (x/x.shift(1) - 1) * 10000)
snapshot["rate2"] = snapshot.groupby("StockID")["ask1p"].apply(lambda x: (x/x.shift(1) - 1) * 10000)

In [185]:
secid = 1

snapshot[(snapshot["StockID"] == secid) & (snapshot["time_interval"] > 60) & ((snapshot["rate1"] > 30)| (snapshot["rate2"] > 30))]


,Date,SENDTIME,StockID,DATATIMESTAMP,PreClosePx,openPrice,HighPx,LowPx,close,NUMTRADES,cum_volume,cum_amount,TOTALLONGPOSITION,PERATIO1,PERATIO2,ENDOFDAYMAKER,TOTALOFFERQTY,WEIGHTEDAVGOFFERPX,TOTALBIDQTY,WEIGHTEDAVGBIDPX,ask10p,ask9p,ask8p,ask7p,ask6p,ask5p,ask4p,ask3p,ask2p,ask1p,bid1p,bid2p,bid3p,bid4p,bid5p,bid6p,bid7p,bid8p,bid9p,bid10p,ask10q,ask9q,ask8q,ask7q,ask6q,ask5q,ask4q,ask3q,ask2q,ask1q,bid1q,bid2q,bid3q,bid4q,bid5q,bid6q,bid7q,bid8q,bid9q,bid10q,NUMORDERS_B1,NOORDERS _B1,ORDERQTY_B1,NUMORDERS_S1,NOORDERS _S1,ORDERQTY_S1,time,OrigTime,cum_max,group,trade_diff,time_diff,volume_update,time1,time2,time3,time4,datetime,time_interval,rate1,rate2


In [187]:
pd.set_option("max_columns", 200)
snapshot[(snapshot["StockID"]==5) & (snapshot["time"] > 140410000)].head(100)

,Date,SENDTIME,StockID,DATATIMESTAMP,PreClosePx,openPrice,HighPx,LowPx,close,NUMTRADES,cum_volume,cum_amount,TOTALLONGPOSITION,PERATIO1,PERATIO2,ENDOFDAYMAKER,TOTALOFFERQTY,WEIGHTEDAVGOFFERPX,TOTALBIDQTY,WEIGHTEDAVGBIDPX,ask10p,ask9p,ask8p,ask7p,ask6p,ask5p,ask4p,ask3p,ask2p,ask1p,bid1p,bid2p,bid3p,bid4p,bid5p,bid6p,bid7p,bid8p,bid9p,bid10p,ask10q,ask9q,ask8q,ask7q,ask6q,ask5q,ask4q,ask3q,ask2q,ask1q,bid1q,bid2q,bid3q,bid4q,bid5q,bid6q,bid7q,bid8q,bid9q,bid10q,NUMORDERS_B1,NOORDERS _B1,ORDERQTY_B1,NUMORDERS_S1,NOORDERS _S1,ORDERQTY_S1,time,OrigTime,cum_max,group,trade_diff,time_diff,volume_update,time1,time2,time3,time4,datetime,time_interval,rate1,rate2
8954,20160329,140416,5,140411180,7.68,7.62,7.7,7.32,7.50,11478,16217967,1.221377e+08,0,132.41,0.0,N,4600922,8.00,3185978,7.24,7.60,7.59,7.58,7.57,7.56,7.55,7.54,7.53,7.52,7.51,7.50,7.49,7.48,7.47,7.46,7.45,7.44,7.43,7.42,7.41,21700,50700,24900,15400,45100,67600,7200,65600,25200,130400,43299,50008,12558,15100,26700,251800,57700,18600,59436,48200,14,14,2599|4000|100|1000|1000|200|1300|300|30000|500...,14,14,300|700|2000|300|200|5600|700|2600|200|500|500...,140411180,20160329140411180,22743378,4680,4.0,3090.0,3401.0,93000320,130000190,150019790.0,140735370,2016-03-29 14:04:11,3.0,0.000000,0.000000
8955,20160329,140740,5,140735370,7.68,7.62,7.7,7.32,7.50,11569,16392067,1.234448e+08,0,132.41,0.0,N,4581522,8.01,3271378,7.24,7.60,7.59,7.58,7.57,7.56,7.55,7.54,7.53,7.52,7.51,7.50,7.49,7.48,7.47,7.46,7.45,7.44,7.43,7.42,7.41,57500,20500,20900,15400,60000,9100,6500,65600,26400,132600,22699,57308,23358,21500,71400,256800,58900,18600,62436,50200,25,25,5199|500|1000|1000|100|200|500|600|1000|500|30...,4,4,16500|500|40000|500,140735370,20160329140735370,22743378,4691,91.0,324190.0,174100.0,93000320,130000190,150019790.0,140735370,2016-03-29 14:07:35,204.0,0.000000,0.000000
8956,20160329,140743,5,140738580,7.68,7.62,7.7,7.32,7.50,11571,16395167,1.234681e+08,0,132.41,0.0,N,4583422,8.01,3268378,7.24,7.60,7.59,7.58,7.57,7.56,7.55,7.54,7.53,7.52,7.51,7.50,7.49,7.48,7.47,7.46,7.45,7.44,7.43,7.42,7.41,59400,20500,20900,15400,60000,9100,6500,65600,26400,132600,19699,57308,23358,21500,71400,256800,58900,18600,62436,50200,25,25,2199|500|1000|1000|100|200|500|600|1000|500|30...,5,5,16400|500|40000|500|2000,140738580,20160329140738580,22743378,4691,2.0,3210.0,3100.0,93000320,130000190,150019790.0,140735370,2016-03-29 14:07:38,3.0,0.000000,0.000000
8957,20160329,140746,5,140741920,7.68,7.62,7.7,7.32,7.50,11571,16395167,1.234681e+08,0,132.41,0.0,N,4583422,8.01,3270878,7.24,7.60,7.59,7.58,7.57,7.56,7.55,7.54,7.53,7.52,7.51,7.50,7.49,7.48,7.47,7.46,7.45,7.44,7.43,7.42,7.41,59400,20500,20900,15400,60000,9100,6500,65600,26400,132600,22199,57308,23358,21500,71400,256800,58900,18600,62436,50200,25,25,2199|500|1000|1000|100|200|500|600|1000|3000|1...,5,5,16400|500|40000|500|2000,140741920,20160329140741920,22743378,4691,0.0,3340.0,0.0,93000320,130000190,150019790.0,140735370,2016-03-29 14:07:41,3.0,0.000000,0.000000
8958,20160329,140749,5,140745010,7.68,7.62,7.7,7.32,7.50,11579,16401167,1.235131e+08,0,132.41,0.0,N,4603922,8.00,3265178,7.24,7.60,7.59,7.58,7.57,7.56,7.55,7.54,7.53,7.52,7.51,7.50,7.49,7.48,7.47,7.46,7.45,7.44,7.43,7.42,7.41,79400,20500,20900,15400,60000,9100,6500,65600,26400,132600,16199,57308,23358,21500,71400,256800,58900,18600,62436,50200,18,18,99|1000|3000|1000|500|300|1000|1500|200|1500|5...,6,6,16400|500|40000|500|2000|20000,140745010,20160329140745010,22743378,4691,8.0,3090.0,6000.0,93000320,130000190,150019790.0,140735370,2016-03-29 14:07:45,4.0,0.000000,0.000000
8959,20160329,140752,5,140747350,7.68,7.62,7.7,7.32,7.50,11581,16401467,1.235153e+08,0,132.41,0.0,N,4603922,8.00,3263478,7.24,7.60,7.59,7.58,7.57,7.56,7.55,7.54,7.53,7.52,7.51,7.50,7.49,7.48,7.47,7.46,7.45,7.44,7.43,7.42,7.41,79400,20500,20900,15400,60000,9100,6500,65600,26400,132600,16899,57308,23358,21500,71400,256800,58900,18600,62436,50200,18,18,799|3000|1000|500|300|1000|1500|200|1500|500|1...,6,6,16400|500|40000|500|2000|200